# Example Usage of BERT Question Answering Model
This notebook demonstrates how to use the BERT Question Answering model with various inputs and settings. The model is pre-trained and fine-tuned on the SQuAD dataset, which makes it suitable for answering questions based on a given passage of text.

## Setup
First, we need to install and import the required libraries.
```python
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
```

## Load the Pre-trained Model and Tokenizer
We'll load the BERT model and tokenizer, which are fine-tuned for question answering.
```python
# Load the pre-trained model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
```

## Define the Question Answering Function
The function `bert_question_answer` will use the BERT model to find the answer to a given question based on the passage.
```python
def bert_question_answer(question, passage, max_len=500):
    # Tokenize the input question and passage
    input_ids = tokenizer.encode(question, passage, max_length=max_len, truncation=True)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    len_question = sep_index + 1
    len_passage = len(input_ids) - len_question
    segment_ids = [0] * len_question + [1] * len_passage
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    start_token_scores = model(torch.tensor([input_ids]))[0].detach().numpy().flatten()
    end_token_scores = model(torch.tensor([input_ids]))[1].detach().numpy().flatten()
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)
    answer = tokens[answer_start_index]
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i].startswith('##'):
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]
    if start_token_scores[answer_start_index] < 0 or answer_start_index == 0 or answer_end_index < answer_start_index or answer == '[SEP]':
        answer = 'Sorry!, I was unable to discover an answer in the passage.'
    return answer
```

## Example 1: Basic Usage
Try answering a question based on a short passage.
```python
# Define a passage and question
passage = "Hello, I am Ishwar. My friend name is Ajay. He is the son of Kristen. I spend most of the time with Ajay."
question = "What is my name"

# Get the answer
answer = bert_question_answer(question, passage)
print(f'Answer: {answer}')
```

## Example 2: Longer Passage
Try answering a question based on a more detailed passage.
```python
# Define a longer passage and question
passage = "Natural language processing, or NLP, is the study of interactions between computers and humans using natural language."
question = "What is NLP"

# Get the answer
answer = bert_question_answer(question, passage)
print(f'Answer: {answer}')
```

## Example 3: Complex Questions
Try answering more complex questions.
```python
# Define a complex passage and question
passage = "Natural language processing (NLP) involves multiple tasks like sentiment analysis, translation, and summarization."
question = "What tasks does NLP involve?"

# Get the answer
answer = bert_question_answer(question, passage)
print(f'Answer: {answer}')
```

## Conclusion
This notebook demonstrated how to use the BERT Question Answering model for different types of questions and passages. Adjust the `max_len` parameter for handling longer passages as needed.